In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats

import os
import subprocess

import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib as ml
import matplotlib.lines as lin
plt = ml.pyplot


In [ ]:
from IPython.core.magic import register_line_magic

@register_line_magic
def shorterr(line):
    """Show only the exception message if one is raised."""
    try:
        output = eval(line)
    except Exception as e:
        print("\x1b[31m\x1b[1m{e.__class__.__name__}: {e}\x1b[0m".format(e=e))
    else:
        return output
    
del shorterr

## Importing data and making DataFrame

In [ ]:

cwd=os.getcwd()
path_data=cwd+'/data/Supernova-chain-15000.txt'

data=np.genfromtxt(path_data)
df=pd.DataFrame(data)

# Eliminating some extra columns 
df.columns=[ 'ones', 'zeros' ,r'$\Omega_m$', 'w', 'chi']
df=df.drop(['ones', 'zeros'], axis=1)
df

## We plot data, begining from minimun of chi column which is best value of parameters

In [ ]:
first_spot=df.iloc[df[['chi']].idxmin()] # We Need ID of Minimum  Point
fp=pd.DataFrame(first_spot)

# Then We Form Parameter Space and Plot "omega_m" and "w" Columns of Minimum  Point

plt.plot( fp[r'$\Omega_m$'], fp['w'], 'ko' )
plt.xlabel(r'$\Omega_m$', fontsize = 18);
plt.ylabel('w', fontsize = 18);

# Best Values of Parameters
fp 

## We import data again to plot 1$\sigma$, 2$\sigma$ and 3$\sigma$ confidence levels

In [ ]:

cwd = os.getcwd()
path_data = cwd+'/data/Supernova-chain-15000.txt'

data = np.genfromtxt(path_data)
da = pd.DataFrame(data)
da.columns = [ 'ones', 'zeros' ,r'$\Omega_m$', 'w', 'chi']
da = da.drop(['ones', 'zeros'], axis =1)

# Forming Subsample_1 to Plot data in 1sigma confidence level (for n= 2 & CL= 68.27%, limit line of 1 sigma CL is chi-chi(min)<2.3)
sbs_1 = da[da.chi-(da['chi'].min()) < 2.3]

plt.rcParams["figure.figsize"] = (5,5)

plt.plot(sbs_1[r'$\Omega_m$'], sbs_1['w'], 'mo', markersize = 1)
plt.plot( fp[r'$\Omega_m$'], fp['w'], 'ko' )
plt.xlabel(r'$\Omega_m$', fontsize = 18);
plt.ylabel('w', fontsize = 18);

# 1 sigma error for "Omega_m" parameter
U_err_om_1 = (sbs_1[r'$\Omega_m$'].max()) - (sbs_1[r'$\Omega_m$'].mean())
D_err_om_1 = (sbs_1[r'$\Omega_m$'].min()) - (sbs_1[r'$\Omega_m$'].mean())

# 1 sigma error for "w" parameter
U_err_w_1 = (sbs_1['w'].max()) - (sbs_1['w'].mean())
D_err_w_1 = (sbs_1['w'].min()) - (sbs_1['w'].mean())



U_err_om_1, D_err_om_1, U_err_w_1, D_err_w_1

In [ ]:
# subsample_1=da[da.chi-(1032.4620111882687)<2.3]
# other_than_subsample_1=da.drop(subsample_1.index)
# db=other_than_subsample_1

# Forming Subsample_2 to Plot data in 2sigma confidence level (for n= 2 & CL= 68.27%, limit line of 2sigma CL is 2.3<chi-chi(min)<6.17)
sbs_2=da[(da.chi - (da['chi'].min()) < 6.17)  &  (2.3 < da.chi - (da['chi'].min()))]


plt.rcParams["figure.figsize"] = (5,5)


plt.plot(sbs_1[r'$\Omega_m$'], sbs_1['w'], 'mo', markersize = 1)
plt.plot(sbs_2[r'$\Omega_m$'], sbs_2['w'], 'yo', markersize = 2)
plt.plot( fp[r'$\Omega_m$'], fp['w'], 'ko' )


plt.xlabel(r'$\Omega_m$', fontsize =18);
plt.ylabel('w', fontsize =18);

# 2 sigma error for "Omega_m" parameter
U_err_om_2 = (sbs_2[r'$\Omega_m$'].max()) - (sbs_2[r'$\Omega_m$'].mean())
D_err_om_2 = (sbs_2[r'$\Omega_m$'].min()) - (sbs_2[r'$\Omega_m$'].mean())

# 2 sigma error for "w" parameter
U_err_w_2 = (sbs_2['w'].max()) - (sbs_2['w'].mean())
D_err_w_2 = (sbs_2['w'].min()) - (sbs_2['w'].mean())

U_err_om_2, D_err_om_2, U_err_w_2, D_err_w_2

In [ ]:

# Forming Subsample_2 to Plot data in 2sigma confidence level (for n= 2 & CL= 68.27%, limit line of 2sigma CL is 6.17<chi-chi(min)<11.8)
sbs_3 = da[(da.chi - (da['chi'].min()) < 11.8)  &  (6.17 < da.chi - (da['chi'].min()))]



plt.plot(sbs_1[r'$\Omega_m$'], sbs_1['w'], 'mo', markersize = 1)
plt.plot(sbs_2[r'$\Omega_m$'], sbs_2['w'], 'co', markersize = 2)
plt.plot(sbs_3[r'$\Omega_m$'], sbs_3['w'], 'bo', markersize = 2)
plt.plot( fp[r'$\Omega_m$'], fp['w'], 'ko', markersize=3 )

plt.xlabel(r'$\Omega_m$', fontsize =18);
plt.ylabel('w', fontsize =18);


# 3 sigma error for "Omega_m" parameter
U_err_om_3 = (sbs_3[r'$\Omega_m$'].max()) - (sbs_3[r'$\Omega_m$'].mean())
D_err_om_3 = (sbs_3[r'$\Omega_m$'].min()) - (sbs_3[r'$\Omega_m$'].mean())

# 3 sigma error for "w" parameter
U_err_w_3 = (sbs_3['w'].max()) - (sbs_3['w'].mean())
D_err_w_3 = (sbs_3['w'].min()) - (sbs_3['w'].mean())

U_err_om_3, D_err_om_3, U_err_w_3, D_err_w_3

## ploting final confidence levels and Gaussian plots of parameters 

In [ ]:

# cwd=os.getcwd()
# path_data=cwd+'/data/Supernova-chain-15000.txt'

# data=np.genfromtxt(path_data)
# dat=pd.DataFrame(data)
# dat.columns=[ 'ones', 'zeros' ,r'$\Omega_m$', 'w', 'chi']
# dat=dat.drop(['ones', 'zeros'], axis=1)


w=da['w']
om=da[r'$\Omega_m$']


fig , axes= plt.subplots (nrows=2 , ncols=2, figsize=(8,8))

# Ploting everythin we have till now, on axes [1,0]
axes[1,0].plot(sbs_1[r'$\Omega_m$'], sbs_1['w'], 'mo', markersize=1)
axes[1,0].plot(sbs_2[r'$\Omega_m$'], sbs_2['w'], 'co', markersize=2)
axes[1,0].plot(sbs_3[r'$\Omega_m$'], sbs_3['w'], 'bo', markersize=2)
axes[1,0].plot(fp[r'$\Omega_m$'], fp['w'], 'ko', markersize=3)
axes[0,1].set_visible(False)


# Ploting Gaussian plot of parameter "w" on axes[1,1] and "omega_m" on axes[0,0]

#axes[1,1].hist(w, bins = 100, color='c', density=True);
ax= sns.kdeplot(w, color='k', bw_adjust=3, ax=axes[1, 1]);

# axes[0,0].hist(omega_m, bins = 100, color='c');
ax1=sns.kdeplot(om, color='k', bw_adjust=3, ax=axes[0,0],  );


axes[1,1].set_xlabel('w', fontsize=15);
axes[1,0].set_ylabel('w', fontsize=15);
axes[1,0].set_xlabel(r'$\Omega_m$', fontsize=15 );
axes[0,0].set_xlabel(' ');
axes[0,0].set_ylabel(' ');
axes[1,1].set_ylabel(' ');


# plt.savefig('contours.eps', bbox_inches="tight", format='eps' )

In [ ]:
from scipy import stats

#0.68 represents 68% confidence level (you can change it to 95.45% and 68.27% too or anything else)
#and n=2 represents the number of free parameters (that can be changed either)

a= stats.chi2.ppf(0.6827, 2)
b= stats.chi2.ppf(0.9545, 2)
c= stats.chi2.ppf(0.9973, 2)
a, b, c

In [ ]:


a= fp['w']
b= fp[r'$\Omega_m$']


data={"err_(+)_om":[U_err_om_1, U_err_om_2, U_err_om_3],
      "err_(-)_om":[D_err_om_1, D_err_om_2, D_err_om_3],
      "err_(+)_w":[U_err_w_1, U_err_w_2, U_err_w_3],
      "err_(-)_w":[D_err_w_1, D_err_w_2, D_err_w_3],
      "best_value": [a],
      "best_value_omega": [b],
     }
bv_table = pd.DataFrame(data, index=['1"$\sigma$"', '2"$\sigma$"',  '3"$\sigma$"'])


bv_table[' best_value_$\Omega$ ']=bv_table.best_value_omega.astype(float)
bv_table[' best_value_w ']=bv_table.best_value.astype(float)

bv_table = bv_table.drop(['best_value_omega', 'best_value'], axis=1)


third_column = bv_table.pop('err_(+)_om')
bv_table.insert(4, 'err_(+)_om', third_column)

third_column = bv_table.pop('err_(-)_om')
bv_table.insert(4, 'err_(-)_om', third_column)



third_column = bv_table.pop('err_(+)_w')
bv_table.insert(5, 'err_(+)_w', third_column)

third_column = bv_table.pop('err_(-)_w')
bv_table.insert(5, 'err_(-)_w', third_column)

bv_table= bv_table.round(decimals=3)

# Saving DataFrame table as csv file
bv_table.to_csv('best_values.csv')

bv_table

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table
import numpy as np



fig, ax = plt.subplots(figsize=(10, 6)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False)  # no visible frame, uncomment if size is ok

tabla = table(ax, bv_table, loc='center', colWidths=[0.17]*len(bv_table.columns))  # where bv_table is your data frame
tabla.auto_set_font_size(False) # Activate set fontsize manually
tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
tabla.scale(1.15,1.15) # change size table


# Saving DataFrame table as figure 
plt.savefig('table.png', transparent=True, facecolor="#FFFFFF")